# Imports

In [1]:
import os
import gym
import wandb
import numpy as np
import time
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Set up Weights & Biases

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cschmidl. Use `wandb login --relogin` to force relogin


True

# Training function for single environment

In [3]:
def train_agent_single_env(config):
    start_time = time.time()

    # Create a gym environment
    env_name = "CartPole-v1"
    env = gym.make(env_name)

    # Wrap the environment in a VecEnv
    env = DummyVecEnv([lambda: env])

    # Set up evaluation environment
    eval_env = gym.make(env_name)
    eval_env = DummyVecEnv([lambda: eval_env])

    # configure the PPo agent
    model = PPO(
        "MlpPolicy",
        env, 
        verbose=1,
        n_steps=config["n_steps"],
        gamma=config["gamma"],
        learning_rate=config["learning_rate"],
        ent_coef=config["ent_coef"],
        clip_range=config["clip_range"],
        n_epochs=config["n_epochs"],
        gae_lambda=config["gae_lambda"],
        max_grad_norm=config["max_grad_norm"]
    )

    # Set up an evaluation callback
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path="./models/",
        log_path="./logs/",
        eval_freq=500,
        deterministic=True,
        render=False
    )

    # Train the PPo agent
    model.learn(total_timesteps=config["total_timesteps"], callback=eval_callback)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Single environment training took {elapsed_time:.2f} seconds.")

    # Evaluate the trained agent
    mean_reward, _ = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

    return mean_reward

# Training function for multi environment

In [4]:
def make_env(env_id, seed):
    def _init():
        env = gym.make(env_id)
        env.seed(seed)
        return env
    return _init

In [15]:
seeds = np.random.randint(0, 2**32 - 1, 4)
print(seeds)

for seed in seeds:
    print(type(seed))

[2848719473  263985200 2543817249 3150862783]
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>


In [16]:
def train_agent_multi_env(config, n_envs=4):
    start_time = time.time()

    # Create a gym environment
    env_name = "CartPole-v1"
    n_envs = n_envs,
    #seeds = np.random.randint(0, int(1e6), n_envs)
    seeds = np.random.randint(0, 2**32 - 1, n_envs)

    envs = [make_env(env_name, int(seed)) for seed in seeds]
    env = SubprocVecEnv(envs)

    # Set up evaluation environment
    eval_env = gym.make(env_name)
    eval_env = DummyVecEnv([lambda: eval_env])

    # Configure the PPO agent
    model = PPO(
        "MlpPolicy",
        env,
        verbose=1,
        n_steps=config["n_steps"],
        gamma=config["gamma"],
        learning_rate=config["learning_rate"],
        ent_coef=config["ent_coef"],
        clip_range=config["clip_range"],
        n_epochs=config["n_epochs"],
        gae_lambda=config["gae_lambda"],
        max_grad_norm=config["max_grad_norm"],
    )

    # Set up an evaluation callback
    eval_callback = EvalCallback(
        eval_env, 
        best_model_save_path="./models/",
        log_path="./logs/",
        eval_freq=500,
        deterministic=True,
        render=False
    )

    # Train the PPO agent
    model.learn(total_timesteps=config["total_timesteps"], callback=eval_callback)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Multi-environment training took {elapsed_time:.2f} seconds.")

    # Evaluate the trained agent
    mean_reward, _ = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)



    return mean_reward

# Set up Weights & Biases sweep

In [17]:
sweep_config_single = {
    "name": "ppo_hyperparameter_tuning_single",
    "method": "random",
    "metric": {"goal": "maximize", "name": "eval/mean_reward"},
    "parameters": {
        "n_steps": {"min": 64, "max": 2048, "distribution": "int_uniform"},
        "gamma": {"min": 0.9, "max": 0.999, "distribution": "uniform"},
        "learning_rate": {"min": 1e-5, "max": 1e-2, "distribution": "uniform"},
        "ent_coef": {"min": 1e-6, "max": 1e-2, "distribution": "uniform"},
        "clip_range": {"min": 0.1, "max": 0.3, "distribution": "uniform"},
        "n_epochs": {"min": 1, "max": 10, "distribution": "int_uniform"},
        "gae_lambda": {"min": 0.9, "max": 1.0, "distribution": "uniform"},
        "max_grad_norm": {"min": 0.1, "max": 10, "distribution": "uniform"},
        "total_timesteps": {"min": 10_000, "max": 100_000, "distribution": "int_uniform"}
    },
}

sweep_config_multi = {
    "name": "ppo_hyperparameter_tuning_multi",
    "method": "random",
    "metric": {"goal": "maximize", "name": "eval/mean_reward"},
    "parameters": {
        "n_steps": {"min": 64, "max": 2048, "distribution": "int_uniform"},
        "gamma": {"min": 0.9, "max": 0.999, "distribution": "uniform"},
        "learning_rate": {"min": 1e-5, "max": 1e-2, "distribution": "uniform"},
        "ent_coef": {"min": 1e-6, "max": 1e-2, "distribution": "uniform"},
        "clip_range": {"min": 0.1, "max": 0.3, "distribution": "uniform"},
        "n_epochs": {"min": 1, "max": 10, "distribution": "int_uniform"},
        "gae_lambda": {"min": 0.9, "max": 1.0, "distribution": "uniform"},
        "max_grad_norm": {"min": 0.1, "max": 10, "distribution": "uniform"},
        "total_timesteps": {"min": 10_000, "max": 100_000, "distribution": "int_uniform"}
    },
}

single_sweep_id = wandb.sweep(sweep_config_single, project="ppo_hyperparamter_tuning")
multi_sweep_id = wandb.sweep(sweep_config_multi, project="ppo_hyperparamter_tuning")

Create sweep with ID: a390ceqc
Sweep URL: https://wandb.ai/cschmidl/ppo_hyperparamter_tuning/sweeps/a390ceqc
Create sweep with ID: b30md251
Sweep URL: https://wandb.ai/cschmidl/ppo_hyperparamter_tuning/sweeps/b30md251


# Define the sweep function

In [13]:
def sweep_agent_single():
    with wandb.init() as run:
        config = run.config
        mean_reward = train_agent_single_env(config)
        wandb.log({"eval/mean_reward": mean_reward})

def sweep_agent_multi():
    with wandb.init() as run:
        config = run.config
        mean_reward = train_agent_multi_env(config, n_envs=4)
        wandb.log({"eval/mean_reward": mean_reward})


# Run the sweep: Single env

In [8]:
# Single
wandb.agent(single_sweep_id, function=sweep_agent_single, count=20)


wandb: Agent Starting Run: ec1zlvay with config:
wandb: 	clip_range: 0.24801498317259676
wandb: 	ent_coef: 0.005259804550102741
wandb: 	gae_lambda: 0.9872416499332052
wandb: 	gamma: 0.9187271553667176
wandb: 	learning_rate: 0.004104321568970994
wandb: 	max_grad_norm: 1.96005489016267
wandb: 	n_epochs: 9
wandb: 	n_steps: 535
wandb: 	total_timesteps: 24494
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Using cpu device


/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/stable_baselines3/ppo/ppo.py:145: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 535`, after every 8 untruncated mini-batches, there will be a truncated mini-batch of size 23
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=535 and n_envs=1)
  warnings.warn(


Eval num_timesteps=500, episode_reward=9.40 +/- 0.80
Episode length: 9.40 +/- 0.80
---------------------------------
| eval/              |          |
|    mean_ep_length  | 9.4      |
|    mean_reward     | 9.4      |
| time/              |          |
|    total_timesteps | 500      |
---------------------------------
New best mean reward!
-----------------------------
| time/              |      |
|    fps             | 1336 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 535  |
-----------------------------


/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


...

Process ForkServerProcess-8:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.13_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python@3.9/3.9.13_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/stable_baselines3/common/vec_env/subproc_vec_env.py", line 25, in _worker
    env = env_fn_wrapper.var()
  File "/var/folders/w3/204l8n9n6g1c3vgzxxhhh7500000gn/T/ipykernel_87305/3773973735.py", line 4, in _init
  File "/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/gym/core.py", line 301, in seed
    return self.env.seed(seed)
  File "/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/gym/envs/classic_control/cartpole.py", line 100,

Run 5jnaseew errored: EOFError()
wandb: ERROR Run 5jnaseew errored: EOFError()
wandb: Agent Starting Run: m7u27qj0 with config:
wandb: 	clip_range: 0.1548381238349878
wandb: 	ent_coef: 0.00863732323016807
wandb: 	gae_lambda: 0.9040056905994543
wandb: 	gamma: 0.9354049976651054
wandb: 	learning_rate: 0.007499545892770019
wandb: 	max_grad_norm: 7.435089408020336
wandb: 	n_epochs: 5
wandb: 	n_steps: 590
wandb: 	total_timesteps: 51464
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Process ForkServerProcess-10:
Process ForkServerProcess-9:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.13_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python@3.9/3.9.13_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/stable_baselines3/common/vec_env/subproc_vec_env.py", line 25, in _worker
    env = env_fn_wrapper.var()
  File "/var/folders/w3/204l8n9n6g1c3vgzxxhhh7500000gn/T/ipykernel_87305/3773973735.py", line 4, in _init
  File "/usr/local/Cellar/python@3.9/3.9.13_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/cschmidl/phd/repos/wandb-playg

Process ForkServerProcess-12:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.13_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python@3.9/3.9.13_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/stable_baselines3/common/vec_env/subproc_vec_env.py", line 25, in _worker
    env = env_fn_wrapper.var()
  File "/var/folders/w3/204l8n9n6g1c3vgzxxhhh7500000gn/T/ipykernel_87305/3773973735.py", line 4, in _init
  File "/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/gym/core.py", line 301, in seed
    return self.env.seed(seed)
  File "/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/gym/envs/classic_control/cartpole.py", line 100

Run m7u27qj0 errored: EOFError()
wandb: ERROR Run m7u27qj0 errored: EOFError()
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


# Run the sweep: Multi env

In [18]:
wandb.agent(multi_sweep_id, function=sweep_agent_multi, count=20)

wandb: Agent Starting Run: cl7g958b with config:
wandb: 	clip_range: 0.2511996313412903
wandb: 	ent_coef: 0.008940710676994899
wandb: 	gae_lambda: 0.948714940686066
wandb: 	gamma: 0.9043587061935928
wandb: 	learning_rate: 0.0004970508894585218
wandb: 	max_grad_norm: 7.593603572713287
wandb: 	n_epochs: 9
wandb: 	n_steps: 179
wandb: 	total_timesteps: 56851
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 3719 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 716  |
-----------------------------


/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/stable_baselines3/ppo/ppo.py:145: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 716`, after every 11 untruncated mini-batches, there will be a truncated mini-batch of size 12
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=179 and n_envs=4)
  warnings.warn(
/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/stable_baselines3/common/callbacks.py:399: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x14267b520> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x14263c310>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


-----------------------------------------
| time/                   |             |
|    fps                  | 2002        |
|    iterations           | 2           |
|    time_elapsed         | 0           |
|    total_timesteps      | 1432        |
| train/                  |             |
|    approx_kl            | 0.008591862 |
|    clip_fraction        | 0.0307      |
|    clip_range           | 0.251       |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00978    |
|    learning_rate        | 0.000497    |
|    loss                 | 6.15        |
|    n_updates            | 9           |
|    policy_gradient_loss | -0.00606    |
|    value_loss           | 22.1        |
-----------------------------------------
Eval num_timesteps=2000, episode_reward=45.80 +/- 6.34
Episode length: 45.80 +/- 6.34
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 45.8        |
|    mean_reward          | 45.8

/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


...

/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/stable_baselines3/ppo/ppo.py:145: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 1380`, after every 21 untruncated mini-batches, there will be a truncated mini-batch of size 36
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=345 and n_envs=4)
  warnings.warn(
/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/stable_baselines3/common/callbacks.py:399: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x142742460> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x142742e50>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


-----------------------------
| time/              |      |
|    fps             | 3301 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 1380 |
-----------------------------


/Users/cschmidl/phd/repos/wandb-playground/venv/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=2000, episode_reward=340.00 +/- 111.86
Episode length: 340.00 +/- 111.86
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 340         |
|    mean_reward          | 340         |
| time/                   |             |
|    total_timesteps      | 2000        |
| train/                  |             |
|    approx_kl            | 0.034938835 |
|    clip_fraction        | 0.234       |
|    clip_range           | 0.289       |
|    entropy_loss         | -0.665      |
|    explained_variance   | 0.013       |
|    learning_rate        | 0.00787     |
|    loss                 | 0.573       |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0316     |
|    value_loss           | 2.89        |
-----------------------------------------
New best mean reward!
-----------------------------
| time/              |      |
|    fps             | 1316 |
|    iterations      | 2    |
|    time_

eval/mean_reward,▁
eval/mean_reward,240.6
